In [72]:
import pandas as pd
import numpy as np

In [6]:
raw_data_folder = "../data/ratings.dat"

In [14]:
ratings = pd.read_csv(raw_data_url, names=["u","v","r"], nrows=1000)

In [50]:
train_data_path =  "../data/matches_train.dat"
test_data_path =  "../data/matches_test.dat"


# freely adapted from https://towardsdatascience.com/beginners-guide-to-creating-an-svd-recommender-system-1fd7326d1f65
# assume ordered ratings so we predict the last ones.
# Corrections made:
# - dummy test and dummy train were interverted.
data = ratings
u = data['u'].unique() #list of all raters
v = data['v'].unique() #list of all ratees
test = pd.DataFrame(columns=data.columns)
train = pd.DataFrame(columns=data.columns)
test_ratio = 0.2 #fraction of data to be used as test set.
for u_id in u:
    temp = data[data['u'] == u_id]
    n = len(temp)
    train_size = int((1-test_ratio)*n)
    
    dummy_train = temp.iloc[:train_size]
    dummy_test = temp.iloc[train_size:]
    
    test = pd.concat([test, dummy_test], ignore_index=True)
    train = pd.concat([train, dummy_train], ignore_index=True)

train.to_csv(train_data_path, index=False)
test.to_csv(test_data_path, index=False)


In [96]:
train.head()

,u,v,r
0,1,133,8
1,1,720,6
2,1,971,10
3,1,1095,7
4,1,1616,10


In [78]:
def create_utility_matrix(data, value_col_name="m"):

    u_field = 0
    v_field = 1
    assert value_col_name in ["m", "r"]
    value_field = 3 if value_col_name == "m" else 2

    u_list = data.iloc[:, u_field].tolist()
    v_list = data.iloc[:, v_field].tolist()
    value_list = data.iloc[:, value_field].tolist()
    u_ids = list(set(data.iloc[:, u_field]))
    v_ids = list(set(data.iloc[:, v_field]))

    u_index = {u_ids[i]: i for i in range(len(u_ids))}
    pd_dict = {v_id: [np.nan for i in range(len(u_ids))] for v_id in v_ids}
    for i in range(0, len(data)):
        v_id = v_list[i]
        u_id = u_list[i]
        value = value_list[i]
        pd_dict[v_id][u_index[u_id]] = value
    X = pd.DataFrame(pd_dict)
    X.index = u_ids

    v_cols = list(X.columns)
    v_index = {v_cols[i]: i for i in range(len(v_cols))}
    # users_index gives us a mapping of user_id to index of user
    # items_index provides the same for items
    return X, u_index, v_index

In [80]:
utility_matrix = create_utility_matrix(train, value_col_name="r")

In [151]:
from functools import partial
import numpy as np
from scipy.linalg import sqrtm
from scipy.sparse.linalg import svds

class FactorizationRecommender:
    def __init__(self, k):
        self.k = k

        self.U = None
        self.s_root = None
        self.V = None
        self.mu_hat = None

        self.rater_index = None
        self.rated_index = None

    # Modified from https://towardsdatascience.com/beginners-guide-to-creating-an-svd-recommender-system-1fd7326d1f65
    # This is not ideal since the traingn
    def fit(self, utility_matrix, rater_index, rated_index):

        # save for use in prediction

        self.rater_index = rater_index
        self.rated_index = rated_index

        k = self.k

        # The magic happens here. U and V are user and item features
        U, s, Vt, mu_hat = self.emsvd(utility_matrix, max_iter=50)

        # we take only the k most significant features
        self.U = U[:, 0:k]
        self.Vt = Vt[0:k, :]
        s = np.diag(s)
        self.s = s
        s = s[0:k, 0:k]
        self.s_root = sqrtm(s)
        # and keep the mean ratings of each raters that were removed
        self.mu_hat = mu_hat

        return self.U, self.s_root, self.Vt, self.mu_hat

    def emsvd(self, utility_matrix, tol=1e-3, max_iter=None):
        """
        Approximate SVD on data with missing values via expectation-maximization

        Inputs:
        -----------
        Y:          (nobs, ndim) data matrix, missing values denoted by NaN/Inf
        tol:        convergence tolerance on change in trace norm
        maxiter:    maximum number of EM steps to perform (default: no limit)

        Returns:
        -----------
        Y_hat:      (nobs, ndim) reconstructed data matrix
        mu_hat:     (ndim,) estimated column means for reconstructed data
        U, s, Vt:   singular values and vectors (see np.linalg.svd and
                    scipy.sparse.linalg.svds for details)
        """
        k = self.k
        svdmethod = partial(svds, k=k)
        if max_iter is None:
            max_iter = np.inf

        # initialize the missing values to their respective column means
        mu_hat = np.nanmean(utility_matrix, axis=0, keepdims=1)
        
        valid = np.isfinite(utility_matrix)
        Y_hat = np.where(valid, utility_matrix, mu_hat)

        halt = False
        ii = 1
        v_prev = 0

        while not halt:
            print(ii)
            # SVD on filled-in data
            U, s, Vt = svdmethod(Y_hat - mu_hat)

            # impute missing values
            Y_hat[~valid] = (U.dot(np.diag(s)).dot(Vt) + mu_hat)[~valid]

            # update bias parameter
            mu_hat = Y_hat.mean(axis=0, keepdims=1)
            print(mu_hat.shape)
            # test convergence using relative change in trace norm
            v = s.sum()
            if ii >= max_iter or ((v - v_prev) / v_prev) < tol:
                halt = True
            ii += 1
            v_prev = v

        return U, s, Vt, mu_hat

    # taken from https://stackoverflow.com/a/35611142/8086033
    def predict(self, rater_id, rated_id):
        # go from id to index
        rater_idx = self.rater_index[rater_id]
        rated_idx = self.rated_index[rated_id]
        # reconstruct the score from decomposed matrix
        u_s_root = np.dot(self.U[rater_idx, :], self.s_root)  # (1,k) array
        s_root_v = np.dot(self.Vt[:, rated_idx], self.s_root) # (k,1)
        score = np.dot(u_s_root, s_root_v)
        return score


    def save(self, path):
        pass

    def load(self, path):
        pass


In [152]:
# emsvd(7, utility_matrix[0], tol=1E-3, maxiter=None)

In [164]:
train

,rater,rated,r,m
0,1,133,8,1.0
1,1,720,6,0.0
2,1,971,10,1.0
3,1,1095,7,0.0
4,1,1616,10,1.0
...,...,...,...,...
10900,100,166185,1,NaN
10901,100,168069,1,NaN
10902,100,171992,10,NaN
10903,100,172954,1,NaN


In [154]:
type_of_value = "r"
k = 10

train = pd.read_csv(train_data_path)

utility_matrix, rater_index, rated_index = create_utility_matrix(train, type_of_value)
fr = FactorizationRecommender(k)
U, s, Vt, mu_hat = fr.fit(utility_matrix, rater_index, rated_index)
print(fr.predict(1,133))

1
(1, 8240)
2
(1, 8240)
3
(1, 8240)
4


C:\Users\charl\miniconda3\envs\d2v_env\lib\site-packages\ipykernel_launcher.py:89: RuntimeWarning: divide by zero encountered in double_scalars


(1, 8240)
5
(1, 8240)
6
(1, 8240)
7
(1, 8240)
8
(1, 8240)
9
(1, 8240)
10
(1, 8240)
11
(1, 8240)
12
(1, 8240)
13
(1, 8240)
14
(1, 8240)
15
(1, 8240)
16
(1, 8240)
17
(1, 8240)
18
(1, 8240)
19
(1, 8240)
20
(1, 8240)
21
(1, 8240)
22
(1, 8240)
23
(1, 8240)
24
(1, 8240)
25
(1, 8240)
26
(1, 8240)
27
(1, 8240)
28
(1, 8240)
29
(1, 8240)
30
(1, 8240)
31
(1, 8240)
32
(1, 8240)
33
(1, 8240)
34
(1, 8240)
35
(1, 8240)
36
(1, 8240)
37
(1, 8240)
38
(1, 8240)
39
(1, 8240)
40
(1, 8240)
41
(1, 8240)
42
(1, 8240)
43
(1, 8240)
44
(1, 8240)
45
(1, 8240)
46
(1, 8240)
47
(1, 8240)
48
(1, 8240)
49
(1, 8240)
50
(1, 8240)
-1.0299029017791197


In [163]:
rater_id = 1
rated_id = 133
rater_idx = fr.rater_index[rater_id]
rated_idx = fr.rated_index[rated_id]
# reconstruct the score from decomposed matrix
u_s_root = np.dot(fr.U[rater_idx, :], fr.s_root)  # (1,k) array
s_root_v = np.dot(fr.Vt[:, rated_idx], fr.s_root) # (k,1)
score = np.dot(u_s_root, s_root_v) +mu_hat[0, rated_idx]
score

8.022964098026534

In [162]:
mu_hat[0][]rated_index

{98304: 0,
 65537: 1,
 2: 2,
 32779: 3,
 32792: 4,
 98334: 5,
 163873: 6,
 65571: 7,
 32804: 8,
 38: 9,
 41: 10,
 32809: 11,
 131122: 12,
 65588: 13,
 55: 14,
 57: 15,
 58: 16,
 61: 17,
 98369: 18,
 65602: 19,
 32840: 20,
 77: 21,
 98381: 22,
 131152: 23,
 84: 24,
 88: 25,
 131161: 26,
 65625: 27,
 131160: 28,
 163943: 29,
 32876: 30,
 65645: 31,
 32884: 32,
 32897: 33,
 65667: 34,
 65668: 35,
 133: 36,
 32902: 37,
 32908: 38,
 163982: 39,
 65679: 40,
 131216: 41,
 145: 42,
 131221: 43,
 153: 44,
 160: 45,
 162: 46,
 164: 47,
 131238: 48,
 131246: 49,
 32953: 50,
 131258: 51,
 131263: 52,
 98509: 53,
 164046: 54,
 131278: 55,
 208: 56,
 211: 57,
 65749: 58,
 131286: 59,
 215: 60,
 98522: 61,
 98523: 62,
 223: 63,
 33006: 64,
 33008: 65,
 131316: 66,
 65786: 67,
 98556: 68,
 98560: 69,
 131329: 70,
 65808: 71,
 98578: 72,
 98581: 73,
 65813: 74,
 65817: 75,
 65827: 76,
 33062: 77,
 297: 78,
 33069: 79,
 65845: 80,
 98615: 81,
 33084: 82,
 65854: 83,
 321: 84,
 328: 85,
 65880: 86,
 6588